# necessary packages

In [ ]:
import csv
import numpy as np
import pandas as pd
import time
from scipy.spatial import distance_matrix
from pyproj import Transformer
import ast
import glob
import os
import seaborn as sns
import matplotlib.pyplot as plt

# get visitation data

read original data

In [ ]:
path1='D:/main/1_research/P10_Mobility/p10_aging mobility/new data/visits/patterns-part1.csv'

col=['placekey',
 'region',
 'poi_cbg',
 'visitor_home_cbgs']
visit_data1=pd.read_csv(path1,encoding='utf-8',skiprows=[708987],usecols=[0,5,14,15])

path2='D:/main/1_research/P10_Mobility/p10_aging mobility/new data/visits/patterns-part2.csv'

col=['placekey',
 'region',
 'poi_cbg',
 'visitor_home_cbgs']
visit_data2=pd.read_csv(path2,encoding='utf-8',usecols=[0,5,14,15])

path3='D:/main/1_research/P10_Mobility/p10_aging mobility/new data/visits/patterns-part3.csv'

visit_data3=pd.read_csv(path3,encoding='utf-8',usecols=[0,5,14,15])

path4='D:/main/1_research/P10_Mobility/p10_aging mobility/new data/visits/patterns-part4.csv'

visit_data4=pd.read_csv(path4,encoding='utf-8',usecols=[0,5,14,15])

visit_data=pd.concat([visit_data1,visit_data2,visit_data3,visit_data4])

#save data
visit_data.to_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/visits/visit_data.csv')

show the combined visitation data

In [3]:
visit_data

,Unnamed: 0,placekey,region,poi_cbg,visitor_home_cbgs
0,0,zzz-222@63f-w7s-rrk,VA,511610303002,"{""517750101003"":4,""517750102001"":4}"
1,1,zzw-223@5zb-xch-7kf,AZ,40131044021,"{""040136117001"":4,""040136183001"":4,""0401310960..."
2,2,225-222@629-3fx-7h5,CT,90035142002,"{""090035148002"":12,""090035142003"":7,""090035141..."
3,3,zzy-222@8sr-8kj-x5z,TX,484790014012,"{""484790017132"":6,""340130091001"":4,""4832395070..."
4,4,22d-222@62j-ss8-3qz,MA,250214042023,"{""090034971001"":5,""250173840023"":4,""2502514020..."
...,...,...,...,...,...
4098730,1122397,zzw-222@5ny-wrv-249,NE,311090005001,"{""310979675001"":4,""311090101001"":4,""3110900130..."
4098731,1122398,225-222@8sx-cbr-g6k,TX,480897503004,"{""480897503004"":12,""480897503003"":11,""48089750..."
4098732,1122399,22c-222@5z5-3mh-dgk,CA,60375340013,"{""060710046015"":4,""060375509024"":4,""0603753480..."
4098733,1122400,232-222@5z6-2mt-zs5,CA,60590423362,"{""060590423242"":5,""060590423331"":5,""0605904233..."


# get census data

In [ ]:
#read original data
cbg_geo_path='D:/main/1_research/P10_Mobility/p10_aging mobility/new data/census/metadata-20231230T041836Z-001/metadata/cbg_geographic_data.csv'
df_census_all=pd.read_csv(cbg_geo_path)

df_census_all[['census_block_group']]=df_census_all[['census_block_group']].astype(str)
#df_cbg_geo['census_block_group']=df_cbg_geo.apply(lambda x: '0'+x['census_block_group'] if len(x['census_block_group'])==11 else x['census_block_group'],axis=1)
df_census_all['state_fips']=df_census_all.apply(lambda x: x['census_block_group'][0:2] if len(x['census_block_group'])==12 else x['census_block_group'][0:1] ,axis=1)

#get population data
cbg_b01_path='D:/main/1_research/P10_Mobility/p10_aging mobility/new data/census/data-20231230T041833Z-001/data/cbg_b01.csv'
df_cbg_b01=pd.read_csv(cbg_b01_path)
list_above_65_id=['B01001e20','B01001e21','B01001e22','B01001e23','B01001e24','B01001e25',
                 'B01001e44','B01001e45','B01001e46','B01001e47','B01001e48','B01001e49']
list_above_50_id=['B01001e16','B01001e17','B01001e18','B01001e19',
                  'B01001e20','B01001e21','B01001e22','B01001e23','B01001e24','B01001e25',
                  'B01001e40','B01001e41','B01001e42','B01001e43',
                 'B01001e44','B01001e45','B01001e46','B01001e47','B01001e48','B01001e49']
population_id='B01001e1'
df_cbg_b01['above_65']=df_cbg_b01[list_above_65_id].sum(axis=1)
df_cbg_b01['above_50']=df_cbg_b01[list_above_50_id].sum(axis=1)
df_cbg_b01=df_cbg_b01[['census_block_group',population_id,'above_65','above_50']]

df_cbg_b01[['census_block_group']]=df_cbg_b01[['census_block_group']].astype(str)
df_census_all=pd.merge(df_census_all,df_cbg_b01)

df_census_all=df_census_all.rename(columns={'B01001e1':'population_by_age','state_fips':'state'})
df_census_all['percent_above_65']=df_census_all.apply(lambda x:x['above_65']/x['population_by_age'] if x['population_by_age']!=0 else 0 ,axis=1)
df_census_all['percent_above_50']=df_census_all.apply(lambda x:x['above_50']/x['population_by_age']if x['population_by_age']!=0 else 0,axis=1)
df_census_all['location']=df_census_all.apply(lambda x: (x['latitude'],x['longitude']),axis=1)

df_county=df_census_all[['census_block_group']]
df_county['county']=df_county.apply(lambda x: x['census_block_group'][0:4] if len(x['census_block_group'])==11 else x['census_block_group'][0:5],axis=1)
df_census_all=pd.merge(df_county,df_census_all)

df_census_all=df_census_all.drop(['amount_land','amount_water','latitude','longitude'],axis=1)

#process race, education and income data

#race='B02001e2'

#process race data
df_race=pd.read_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/census/data-20231230T041833Z-001/data/cbg_b02.csv') 

df_race['percent_minority']=df_race.apply(lambda x:1-x['B02001e2']/x['B02001e1'] if x['B02001e1']!=0 else 2,axis=1)
df_race=df_race[['census_block_group','percent_minority']]



#process education data #'percent_undergraduate or higher'
list_undergraduate_or_higher=['B14002e19','B14002e22','B14002e43','B14002e46']

df_educ=pd.read_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/census/data-20231230T041833Z-002/data/cbg_b14.csv') 

df_educ['num_undergraduate or higher']=df_educ[list_undergraduate_or_higher].sum(axis=1)
df_educ=df_educ[['census_block_group','num_undergraduate or higher']]

#process income data #med_income='B19013e1'
df_med_income=pd.read_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/census/data-20231230T041833Z-002/data/cbg_b19.csv')
df_med_income['median_income_normalized by 250000']=df_med_income.apply(lambda x: x['B19013e1']/250000,axis=1)

df_med_income=df_med_income[['census_block_group','median_income_normalized by 250000']]

df_med_income['census_block_group']=df_med_income['census_block_group'].astype(str)
df_educ['census_block_group']=df_educ['census_block_group'].astype(str)
df_race['census_block_group']=df_race['census_block_group'].astype(str)

df_census_all=pd.merge(df_census_all,df_med_income,how='left')
df_census_all=pd.merge(df_census_all,df_educ,how='left')
#df_census_all=pd.merge(df_census_all,df_race,how='left')

df_census_all['percent_undergraduate or higher']=df_census_all.apply(lambda x: x['num_undergraduate or higher']/x['population_by_age'] if x['population_by_age']!=0 else 2,axis=1)
df_census_all=df_census_all.drop(['num_undergraduate or higher'],axis=1)
df_census_all=pd.merge(df_census_all,df_race,how='left')

df_census_all['median_income_normalized by 250000'].fillna(2,inplace=True)

#save census data
df_census_all.to_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/census/all_census_data_with_location.csv')

# get POI data

In [ ]:
#read original data
df_place=pd.DataFrame()
for i in range(6):
    path='D:/main/1_research/P10_Mobility/p10_aging mobility/new data/Places/core_poi-part'+str(i+1)+'.csv'
    df_p=pd.read_csv(path)
    df_place=pd.concat([df_place,df_p])
    print(len(df_p))

#save data
df_place.to_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/Places/all_places.csv')

state epsg

In [ ]:
dict_epsg={'1': '32616', '2': '32606', '4': '32612', '5': '32615', '6': '32611', '8': '32613', '9': '32618', '10': '32618', '11': '32618', '12': '32617', '13': '32616', '15': '32604', '16': '32611', '17': '32616', '18': '32616', '19': '32615', '20': '32614', '21': '32616', '22': '32615', '23': '32619', '24': '32618', '25': '32619', '26': '32617', '27': '32615', '28': '32616', '29': '32615', '30': '32612', '31': '32614', '32': '32611', '33': '32619', '34': '32618', '35': '32613', '36': '32618', '37': '32617', '38': '32614', '39': '32617', '40': '32614', '41': '32610', '42': '32618', '44': '32619', '45': '32617', '46': '32614', '47': '32616', '48': '32614', '49': '32612', '50': '32618', '51': '32618', '53': '32610', '54': '32617', '55': '32616', '56': '32613', '72': '32619'}

# calculate A

get supply set

In [ ]:
def axis_convert(loc,epsg):
    # lat=loc[0]
    # lon=loc[1]
    transformer=Transformer.from_crs("epsg:4326", "epsg:"+epsg)#WGS 84 / UTM zone 17N #florida
    x,y=transformer.transform(loc[0], loc[1])
    return [x,y]

def get_supply_set(state,state_fip,epsg,df_census_with_location,places0,place_naics_code_dict,p_out):
    print('step 0: read data and preparation')
    
    #read input data
    df_census_with_location1=df_census_with_location[df_census_with_location['state']==int(state_fip)]
    df_census_with_location1=df_census_with_location1.drop(['Unnamed: 0'],axis=1) 
    
    # Read places data with placekey and naics codes
    places = places0[places0['region']==state]
    
    #transform location
    
    df_census_with_location1['trans_loc']=df_census_with_location1.apply(lambda x: axis_convert(eval(x['location']),epsg),axis=1)
    
    # Deal with noices
    places['naics_code'].replace([np.inf, -np.inf], 0, inplace=True)
    places=places[places['naics_code'].notna()]
    places['naics_code'] = places['naics_code'].astype(int)
    places['naics_code'] = places['naics_code'].astype(str)
    
    places['2_naics_code']=places.apply(lambda x: x.naics_code[0:2],axis=1)
    places['3_naics_code']=places.apply(lambda x: x.naics_code[0:3],axis=1) 
    places['4_naics_code']=places.apply(lambda x: x.naics_code[0:4],axis=1) 
    places['5_naics_code']=places.apply(lambda x: x.naics_code[0:5],axis=1) 
    
    
    z0=np.array(df_census_with_location1['trans_loc'].to_list()) #block poi

    #get place and location data
    plc={}
    for key in place_naics_code_dict.keys():  
        
        naics_codes=place_naics_code_dict[key][0]
        naics_level=place_naics_code_dict[key][2]
        reduced_naics_code=str(naics_level)+'_naics_code'
        places_ = places.loc[places[reduced_naics_code]==naics_codes]
        
        print("There are %s places related to "%len(places_) +key)
        
        if len(places_)>0: 
            
            print(key+' added transformed locations of places')
            pl={}
            for i,pk in enumerate(places_['placekey'].to_list()):
                pl[pk]=axis_convert([places_[places_['placekey']==pk]['latitude'].iat[0],places_[places_['placekey']==pk]['longitude'].iat[0]],epsg)
                #print (i)
            print('step 1: start calculating distance_matrix')
            z1=np.array(list(pl.values())) #place poi
            d_matrix=distance_matrix(z0,z1,p=2)
            #dist_matri_dic[key]=d_matrix

            print('step 2: get supply')
            place_tem=pd.DataFrame(pd.Series(pl),columns=['trans_loc']).reset_index().rename(columns={'placekey':'trans_loc'})
            
            d_cap=place_naics_code_dict[key][1]*1.609344 #10miles
            homo_loca_acc_fac=[]
            hete_loca_acc_fac=[]
            for i,bg in enumerate(df_census_with_location1['census_block_group'].to_list()):
                #loc0=eval(df_census_with_location[df_census_with_location['census_block_group']==bg]['location'].iat[0])
                
                homo_supply_set=[]
                hete_supply_set=[]
                print(i)
                for j,d in enumerate(d_matrix[i,:]):
                    if d<=d_cap/2: #d=5 miles
                        homo_supply_set.append(place_tem.iloc[j,0])
                    if d<=d_cap:
                        hete_supply_set.append(place_tem.iloc[j,0])
                
                homo_loca_acc_fac.append([bg,key,homo_supply_set,len(homo_supply_set)])#for each child type    
                
                hete_loca_acc_fac.append([bg,key,hete_supply_set,len(hete_supply_set)])#for each child type               
                
            df_loca=pd.DataFrame(homo_loca_acc_fac,columns=['census_block_group','social_child_type','accessible_facility_set','num_accessible_facility'])
            path_l=p_out+'/homo_supply only/'+key+'.csv'
            df_loca.to_csv(path_l)
            print(key+' homo done '+state)
        
            
            df_loca=pd.DataFrame(hete_loca_acc_fac,columns=['census_block_group','social_child_type','accessible_facility_set','num_accessible_facility'])
            path_l=p_out+'/hete_supply only/'+key+'.csv'
            df_loca.to_csv(path_l)
            print(key+' hete done '+state)           


        else:
            zero_supply_list=[]
            for i,bg in enumerate(df_census_with_location1['census_block_group'].to_list()):
                zero_supply_list.append([bg,key,[],0])
            df_loca=pd.DataFrame(zero_supply_list,columns=['census_block_group','social_child_type','accessible_facility_set','num_accessible_facility'])
            path_l=p_out+'/homo_supply only/'+key+'.csv'
            df_loca.to_csv(path_l)
            
            path_l=p_out+'/hete_supply only/'+key+'.csv'
            df_loca.to_csv(path_l)


In [ ]:
if __name__=='__main__':
    place_naics_code_dict={'Ambulatory Health Care Services':['621',10000,3,'Health Care Services'],
                            'Hospitals':['622', 10000,3,'Health Care Services'],
                            'Nursing and Residential Care Facilities':['623',10000,3,'Health Care Services'],
                            'Pharmacies and Drug Stores':['44611',10000,5,'Pharmaceutical and Medical Supply Stores'],
                            'Grocery Stores':['4451',10000,4,'Grocery and Food Supply'],
                            'Specialty Food Stores':['4452', 10000,4,'Grocery and Food Supply'],
                            'Individual and Family Services':['6241', 10000,4,'Social Assistance'],
                            'Services for the Elderly and Persons with Disabilities':['62412',10000,5,'Social Assistance'],
                            'Transit and Ground Passenger Transportation':['485', 10000,3,'Transportation Services'],
                            'Taxi and Limousine Service':['48531', 10000,5,'Transportation Services'],
                            'Nonemergency Medical Transportation':['48532', 10000,5,'Transportation Services'],
                            'Home Health Care Services':['6216', 10000,4,'Home Health Care Services'],
                            'Credit Intermediation and Related Activities':['522',10000,3,'Financial Services'],
                            'Securities, Commodity Contracts, and Other Financial Investments and Related Activities':['523', 10000,3,'Financial Services'],
                            'Legal Services':['5411',10000,4,'Legal Services'],
                            'Real Estate':['531', 10000,3,'Housing and Real Estate'],
                            'Real Estate Property Managers':['53131', 10000,5,'Housing and Real Estate'],
                            'Personal Care Services':['8121', 10000,4,'Personal Care Services'],
                            'Utilities':['22', 10000,2,'Utility Services'],
                            'Clothing and Clothing Accessories Stores':['448',10000,3,'Retail Stores for Basic Needs'],
                            'Furniture Stores':['442',10000,3,'Retail Stores for Basic Needs'],
                            'Electronics and Appliance Stores':['443',10000,3,'Retail Stores for Basic Needs']} 
    
    dict_epsg={'1': '32616', '2': '32606', '4': '32612', '5': '32615', '6': '32611', '8': '32613', '9': '32618', '10': '32618', '11': '32618', '12': '32617', 
               '13': '32616', '15': '32604', '16': '32611', '17': '32616', '18': '32616', '19': '32615', '20': '32614', '21': '32616', '22': '32615', '23': '32619', 
               '24': '32618', '25': '32619', '26': '32617', '27': '32615', '28': '32616', '29': '32615', '30': '32612', '31': '32614', '32': '32611', '33': '32619', 
               '34': '32618', '35': '32613', '36': '32618', '37': '32617', '38': '32614', '39': '32617', '40': '32614', '41': '32610', '42': '32618', '44': '32619', 
               '45': '32617', '46': '32614', '47': '32616', '48': '32614', '49': '32612', '50': '32618', '51': '32618', '53': '32610', '54': '32617', '55': '32616', 
               '56': '32613', '72': '32619'}
    
    dict_state={'1': 'AL', '2': 'AK', '4': 'AZ', '5': 'AR', '6': 'CA', '8': 'CO', '9': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID',
                '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO',
                '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', 
                '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '60': 'AS', 
                '66': 'GU', '69': 'MP', '72': 'PR', '74': 'UM', '78': 'VI'}
    
    done_list=['1', '2', '4', '5', '6', '8', '9', '10', '11', '12', '13', '15', '16','17', 
               '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29','30', 
               '31', '32', '33', '34', '35','36']
    df_census_with_location=pd.read_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/census/all_census_data_with_location.csv')
    places=pd.read_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/Places/all_places.csv')

    for fip in dict_epsg.keys():
        if fip not in done_list:
            state_fip=fip
            state=dict_state[state_fip]
            epsg= dict_epsg[state_fip]
            print(state +'start')

            #create folder
            main_path='D:/main/1_research/P10_Mobility/p10_aging mobility/results_all_states/'
            p_out=main_path+state+'/output'
            p_out_1=main_path+state+'/output/homo_supply only'
            p_out_2=main_path+state+'/output/hete_supply only'
            p_out_3=main_path+state+'/output/homo_supply_demand'
            p_out_4=main_path+state+'/output/hete_supply_demand'
            list_path=[p_out,p_out_1,p_out_2,p_out_3,p_out_4]
            for ph in list_path:
                if not os.path.exists(ph):
                    os.makedirs(ph)

            get_supply_set(state,state_fip,epsg,df_census_with_location,places,place_naics_code_dict,p_out)
            print(state +'end')

match supply and demand

In [ ]:
def homo_match_supply_demand (state,state_fip,df_census_with_location,df_visitation_data,place_naics_code_dict,p_out):
    
    df_social= df_visitation_data[df_visitation_data['state']==state_fip]
    
    df1 =df_census_with_location[df_census_with_location['state']==int(state_fip)]
    
    df1['C'] = ['greater than 0.5' if i > 0.5 else 'less than or equal to 0.5' for i in df1['percent_above_65']]
    
    print('start iteration')
    for key in place_naics_code_dict.keys():
        
        path_l=p_out+'/homo_supply only/'+key+'.csv'
        
        df_sup= pd.read_csv(path_l)
        df_sup=df_sup.drop(['Unnamed: 0'],axis=1)
    
     
        demand=[]    
        ratio_demand=[]
        age=[]
        met_in_local=[]
        for i,bg in enumerate(df_sup['census_block_group'].to_list()):
            demand.append(sum(df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['no_visits']))
            ratio_demand.append(sum(df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['ratio']))
            age.append(df1[df1['census_block_group']==bg]['C'].iat[0])
            if sum(df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['no_visits'])>0:          
                k=0
                for plc in df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['placekey'].to_list():
                    if plc in df_sup[(df_sup['social_child_type']==key)&(df_sup['census_block_group']==bg)]['accessible_facility_set'].iat[0]:
                        k=k+df_social[(df_social['placekey']==plc)&(df_social['census_block_group']==bg)]['no_visits'].iat[0]
                met_in_local.append(k/sum(df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['no_visits']))
            else:
                met_in_local.append(2) 
            print(str(i)+'_'+key)
        
        df_sup['demand_visits']=demand
      
        df_sup['demand_ratios']=ratio_demand
        df_sup['demand_met_in_local']=met_in_local
    
        df_sup['age']=age
    
        path_l=p_out+'/homo_supply_demand/'+key+'.csv'
        df_sup.to_csv(path_l)
        print(key+' homo done')        

def hete_match_supply_demand(state,state_fip,df_census_with_location,df_visitation_data,place_naics_code_dict,p_out):
    df_social= df_visitation_data[df_visitation_data['state']==state_fip]
    
    df1 =df_census_with_location[df_census_with_location['state']==int(state_fip)]
    
    df1['C'] = ['greater than 0.5' if i > 0.5 else 'less than or equal to 0.5' for i in df1['percent_above_65']]
    
   
    
    print('start iteration')
    for key in place_naics_code_dict.keys():
        
        path_l=p_out+'/hete_supply only/'+key+'.csv'
        
        df_sup= pd.read_csv(path_l)
        df_sup=df_sup.drop(['Unnamed: 0'],axis=1)
    
     
        demand=[]    
        ratio_demand=[]
        age=[]
        met_in_local=[]
        for i,bg in enumerate(df_sup['census_block_group'].to_list()):
            demand.append(sum(df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['no_visits']))
            ratio_demand.append(sum(df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['ratio']))
            age.append(df1[df1['census_block_group']==bg]['C'].iat[0])
            if sum(df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['no_visits'])>0:          
                k=0
                for plc in df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['placekey'].to_list():
                    if plc in df_sup[(df_sup['social_child_type']==key)&(df_sup['census_block_group']==bg)]['accessible_facility_set'].iat[0]:
                        k=k+df_social[(df_social['placekey']==plc)&(df_social['census_block_group']==bg)]['no_visits'].iat[0]
                met_in_local.append(k/sum(df_social[(df_social['child_type']==key)&(df_social['census_block_group']==bg)]['no_visits']))
            else:
                met_in_local.append(2) 
            print(str(i)+'_'+key)
        
        df_sup['demand_visits']=demand
      
        df_sup['demand_ratios']=ratio_demand
        df_sup['demand_met_in_local']=met_in_local
    
        df_sup['age']=age
    
        path_l=p_out+'/hete_supply_demand/'+key+'.csv'
        df_sup.to_csv(path_l)
        print(key+' hete done') 
        

In [ ]:
if __name__=='__main__':
    place_naics_code_dict={'Ambulatory Health Care Services':['621',10000,3,'Health Care Services'],
                            'Hospitals':['622', 10000,3,'Health Care Services'],
                            'Nursing and Residential Care Facilities':['623',10000,3,'Health Care Services'],
                            'Pharmacies and Drug Stores':['44611',10000,5,'Pharmaceutical and Medical Supply Stores'],
                            'Grocery Stores':['4451',10000,4,'Grocery and Food Supply'],
                            'Specialty Food Stores':['4452', 10000,4,'Grocery and Food Supply'],
                            'Individual and Family Services':['6241', 10000,4,'Social Assistance'],
                            'Services for the Elderly and Persons with Disabilities':['62412',10000,5,'Social Assistance'],
                            'Transit and Ground Passenger Transportation':['485', 10000,3,'Transportation Services'],
                            'Taxi and Limousine Service':['48531', 10000,5,'Transportation Services'],
                            'Nonemergency Medical Transportation':['48532', 10000,5,'Transportation Services'],
                            'Home Health Care Services':['6216', 10000,4,'Home Health Care Services'],
                            'Credit Intermediation and Related Activities':['522',10000,3,'Financial Services'],
                            'Securities, Commodity Contracts, and Other Financial Investments and Related Activities':['523', 10000,3,'Financial Services'],
                            'Legal Services':['5411',10000,4,'Legal Services'],
                            'Real Estate':['531', 10000,3,'Housing and Real Estate'],
                            'Real Estate Property Managers':['53131', 10000,5,'Housing and Real Estate'],
                            'Personal Care Services':['8121', 10000,4,'Personal Care Services'],
                            'Utilities':['22', 10000,2,'Utility Services'],
                            'Clothing and Clothing Accessories Stores':['448',10000,3,'Retail Stores for Basic Needs'],
                            'Furniture Stores':['442',10000,3,'Retail Stores for Basic Needs'],
                            'Electronics and Appliance Stores':['443',10000,3,'Retail Stores for Basic Needs']} 
    
    dict_epsg={'1': '32616', '2': '32606', '4': '32612', '5': '32615', '6': '32611', '8': '32613', '9': '32618', '10': '32618', '11': '32618', '12': '32617', 
               '13': '32616', '15': '32604', '16': '32611', '17': '32616', '18': '32616', '19': '32615', '20': '32614', '21': '32616', '22': '32615', '23': '32619', 
               '24': '32618', '25': '32619', '26': '32617', '27': '32615', '28': '32616', '29': '32615', '30': '32612', '31': '32614', '32': '32611', '33': '32619', 
               '34': '32618', '35': '32613', '36': '32618', '37': '32617', '38': '32614', '39': '32617', '40': '32614', '41': '32610', '42': '32618', '44': '32619', 
               '45': '32617', '46': '32614', '47': '32616', '48': '32614', '49': '32612', '50': '32618', '51': '32618', '53': '32610', '54': '32617', '55': '32616', 
               '56': '32613', '72': '32619'}
    
    dict_state={'1': 'AL', '2': 'AK', '4': 'AZ', '5': 'AR', '6': 'CA', '8': 'CO', '9': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID',
                '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO',
                '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', 
                '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '60': 'AS', 
                '66': 'GU', '69': 'MP', '72': 'PR', '74': 'UM', '78': 'VI'}
 
    df_census_with_location=pd.read_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/census/all_census_data_with_location.csv')
    places=pd.read_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/Places/all_places.csv')
    df_visitation_data=pd.read_csv('D:/main/1_research/P10_Mobility/p10_aging mobility/new data/visitation_data.csv')
    df_visitation_data['census_block_group'] =df_visitation_data['census_block_group'].astype(str)
    df_visitation_data['state']=df_visitation_data.apply(lambda x:x['census_block_group'][0:2] 
                                                         if len(x['census_block_group'])==12 else x['census_block_group'][0:1],axis=1)
    

    state_fip='1'
    state=dict_state[state_fip]
    epsg= dict_epsg[state_fip]
    
    
    #create folder
    main_path='D:/main/1_research/P10_Mobility/p10_aging mobility/results_all_states/'
    p_out=main_path+state+'/output'
    p_out_1=main_path+state+'/output/homo_supply only'
    p_out_2=main_path+state+'/output/hete_supply only'
    p_out_3=main_path+state+'/output/homo_supply_demand'
    p_out_4=main_path+state+'/output/hete_supply_demand'
    list_path=[p_out,p_out_1,p_out_2,p_out_3,p_out_4]
    # for ph in list_path:
    #     if not os.path.exists(ph):
    #         os.makedirs(ph)
    
    homo_match_supply_demand (state,state_fip,df_census_with_location,df_visitation_data,place_naics_code_dict,p_out)

In [ ]:
#final outcomes are A of all CBGs across the US in both 5-mile and 10-mile settings. 